# Preamble

In [2]:
%load_ext autoreload

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import torch
from tqdm import tqdm
from sklearn import metrics

import config

In [3]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True


# Validating

In [7]:
def reportScore(y_true, y_pred):
    print("\t\tAccuracy:\t" + str(round(metrics.accuracy_score(y_true,y_pred), 2)))
    print("\t\tPrecision:\t" + str(round(metrics.precision_score(y_true,y_pred), 2)))
    print("\t\tRecall:   \t" + str(round(metrics.recall_score(y_true,y_pred), 2)))
    print("\t\tF1-score:\t" + str(round(metrics.f1_score(y_true,y_pred), 2)))
    #tn, fp, fn, tp = metrics.confusion_matrix(y_true,y_pred).ravel()

In [8]:
def validate(model, data_split="validation", progress_bar=True):
    data_loader = model.get_dataloader(data_split, shuffle=False) 
    model.eval()

    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in tqdm(data_loader,disable=not progress_bar):
            data['audio'], data['label'] = data['audio'].to(device), data['label'].to(device)
            outputs = model(data)

            y_pred.append(torch.max(outputs.data, 1)[1].item())
            y_true.append(data['label'].item())
            
    reportScore(y_true, y_pred)

In [9]:
def load_models(models):
    loades_models = []
    
    for (model_path, model_class) in models:
        loaded_dict = torch.load(model_path)
        model = model_class(loaded_dict['hparams'])
        model.model.load_state_dict(loaded_dict["state_dict"])
        model = model.eval().cuda()
        loades_models.append(model)
    return loades_models

# Dataset 1

In [34]:
from datasets.datasethandler import DatasetHandler 
datasetHandler = DatasetHandler()

In [35]:
from classification.models.M5 import M5PLModule
from classification.models.CRNN import CRNNPLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule

# 8k models
from classification.models.SpectrogramCNN_8K import SpectrogramCNN_8KPLModule
from classification.models.DeepRecursiveCNN8k import DeepRecursiveCNN8kPLModule
from classification.models.CRNN_8k import CRNN8kPLModule

In [49]:
models = [# 48k models
          ('/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN/SpectrogramCNN_optuna_90.5.p', SpectrogramCNNPLModule),
          ("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/DeepRecursiveCNN/DeepRecursiveCNN_vanilla.p", DeepRecursiveCNNPLModule),
          ("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/CRNN/vanilla_0.878.p", CRNNPLModule),
          
          # 8k models
          ("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_vanilla.p", M5PLModule),
          ("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN_8K/SpectrogramCNN_8k_vanilla_0.8998.p", SpectrogramCNN_8KPLModule),
          ("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/CRNN8k/CRNN8k_vanilla_0.8951.p", CRNN8kPLModule),
          ("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/DeepRecursiveCNN8k/optuna_deep8k_0.87.p", DeepRecursiveCNN8kPLModule)
         ]
models = load_models(models)

In [13]:
for model in models:
    datasetHandler.load_datasets(model, old_data=(model.__class__.__name__ == "M5PLModule"))

In [14]:
for model in models:
    print(model.__class__.__name__)
    for data_split in ["training", "validation", "testing"]:
        print("\t" + data_split)
        validate(model, data_split=data_split, progress_bar=False)
    print()

SpectrogramCNNPLModule
	training
		Accuracy:	0.94
		Precision:	0.94
		Recall:   	0.95
		F1-score:	0.94
	validation
		Accuracy:	0.9
		Precision:	0.9
		Recall:   	0.91
		F1-score:	0.9
	testing
		Accuracy:	0.9
		Precision:	0.9
		Recall:   	0.89
		F1-score:	0.9

DeepRecursiveCNNPLModule
	training
		Accuracy:	0.88
		Precision:	0.88
		Recall:   	0.89
		F1-score:	0.88
	validation
		Accuracy:	0.89
		Precision:	0.88
		Recall:   	0.9
		F1-score:	0.89
	testing
		Accuracy:	0.89
		Precision:	0.89
		Recall:   	0.88
		F1-score:	0.88

CRNNPLModule
	training
		Accuracy:	0.88
		Precision:	0.88
		Recall:   	0.89
		F1-score:	0.89
	validation
		Accuracy:	0.88
		Precision:	0.87
		Recall:   	0.88
		F1-score:	0.88
	testing
		Accuracy:	0.87
		Precision:	0.88
		Recall:   	0.86
		F1-score:	0.87

M5PLModule
	training
		Accuracy:	0.89
		Precision:	0.88
		Recall:   	0.91
		F1-score:	0.89
	validation
		Accuracy:	0.87
		Precision:	0.86
		Recall:   	0.88
		F1-score:	0.87
	testing
		Accuracy:	0.88
		Precision:	0.88
		R

# Dataset 2

In [6]:
from datasets.datasethandler import DatasetHandler 
datasetHandler_dataset2 = DatasetHandler()

In [7]:
from classification.models.SpectrogramCNN_8K_Dataset2 import SpectrogramCNN_8K_Dataset2_PLModule
from classification.models.CRNN_8k_D2 import CRNN8k_D2_PLModule

In [10]:
models = [
          ("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN8K_D2/optuna_spec8kd2_0.9144.p", SpectrogramCNN_8K_Dataset2_PLModule),
          ("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/CRNN8k_D2/optuna_crnn8kd2_0.897.p", CRNN8k_D2_PLModule)
         ]
models = load_models(models)

In [11]:
for model in models:
    datasetHandler_dataset2.load_datasets(model, dataset_id=config.DATASET_CONTROL)

Loading cached training data of dataset 1 from /nfs/students/summer-term-2020/project-4/data/dataset2/dataset_8k/
Loading cached validation data of dataset 1 from /nfs/students/summer-term-2020/project-4/data/dataset2/dataset_8k/
Loading cached testing data of dataset 1 from /nfs/students/summer-term-2020/project-4/data/dataset2/dataset_8k/


In [16]:
for model in models:
    print(model.__class__.__name__)
    for data_split in ["training", "validation", "testing"]:
        print("\t" + data_split)
        validate(model, data_split=data_split, progress_bar=False)
    print()

SpectrogramCNN_8K_Dataset2_PLModule
	training
		Accuracy:	0.94
		Precision:	0.94
		Recall:   	0.95
		F1-score:	0.95
	validation
		Accuracy:	0.91
		Precision:	0.9
		Recall:   	0.93
		F1-score:	0.92
	testing
		Accuracy:	0.91
		Precision:	0.9
		Recall:   	0.92
		F1-score:	0.91

CRNN8k_D2_PLModule
	training
		Accuracy:	0.93
		Precision:	0.94
		Recall:   	0.91
		F1-score:	0.93
	validation
		Accuracy:	0.9
		Precision:	0.91
		Recall:   	0.88
		F1-score:	0.89
	testing
		Accuracy:	0.89
		Precision:	0.9
		Recall:   	0.87
		F1-score:	0.89

